In [1]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [4]:
#Define input image dimensions

img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

def build_generator():

    noise_shape = (100,) #1D array of size 100 (latent vector / noise)

#Define your generator network 
#Here we are only using Dense layers. But network can be complicated based
#on the application. For example, you can use VGG for super res. GAN.         

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)    #Generated image

    return Model(noise, img)

#Alpha — α is a hyperparameter which controls the underlying value to which the
#function saturates negatives network inputs.
#Momentum — Speed up the training
##########################################################################

#Given an input image, the Discriminator outputs the likelihood of the image being real.
#Binary classification - true or false 

def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)
#The validity is the Discriminator’s guess of input being real or not.

In [8]:
#Now that we have constructed our two models it’s time to pit them against each other.
#We do this by defining a training function, loading the data set, re-scaling our training
#images and setting the ground truths. 
def train(epochs, batch_size=128, save_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Convert to float and Rescale -1 to 1 (Can also do 0 to 1)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5

#Add channels dimension. As the input to our gen and discr. has a shape 28x28x1.
    X_train = np.expand_dims(X_train, axis=3) 

    half_batch = int(batch_size / 2)


#We then loop through a number of epochs to train our Discriminator by first selecting
#a random batch of images from our true dataset, generating a set of images from our
#Generator, feeding both set of images into our Discriminator, and finally setting the
#loss parameters for both the real and fake images, as well as the combined loss. 
    
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]

 
        noise = np.random.normal(0, 1, (half_batch, 100))

        # Generate a half batch of fake images
        gen_imgs = generator.predict(noise)

        # Train the discriminator on real and fake images, separately
        #Research showed that separate training is more effective. 
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    #take average loss from real and fake images. 
    #
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 

#And within the same loop we train our Generator, by setting the input noise and
#ultimately training the Generator to have the Discriminator label its samples as valid
#by specifying the gradient loss.
        # ---------------------
        #  Train Generator
        # ---------------------
#Create noise vectors as input for generator. 
#Create as many noise vectors as defined by the batch size. 
#Based on normal distribution. Output will be of size (batch size, 100)
        noise = np.random.normal(0, 1, (batch_size, 100)) 

        # The generator wants the discriminator to label the generated samples
        # as valid (ones)
        #This is where the genrator is trying to trick discriminator into believing
        #the generated image is true (hence value of 1 for y)
        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size

        # Generator is part of combined where it got directly linked with the discriminator
        # Train the generator with noise as x and 1 as y. 
        # Again, 1 as the output as it is adversarial and if generator did a great
        #job of folling the discriminator then the output would be 1 (true)
        g_loss = combined.train_on_batch(noise, valid_y)


#Additionally, in order for us to keep track of our training process, we print the
#progress and save the sample image output depending on the epoch interval specified.  
# Plot the progress
        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch)

#when the specific sample_interval is hit, we call the
#sample_image function. Which looks as follows.

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()
#This function saves our images for us to view


##############################################################################

#Let us also define our optimizer for easy use later on.
#That way if you change your mind, you can change it easily here
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.

# Build and compile the discriminator first. 
#Generator will be trained as part of the combined model, later. 
#pick the loss function and the type of metric to keep track.                 
#Binary cross entropy as we are doing prediction and it is a better
#loss function compared to MSE or other. 
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

#build and compile our Discriminator, pick the loss function

#SInce we are only generating (faking) images, let us not track any metrics.
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

##This builds the Generator and defines the input noise. 
#In a GAN the Generator network takes noise z as an input to produce its images.  
z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)

#This ensures that when we combine our networks we only train the Generator.
#While generator training we do not want discriminator weights to be adjusted. 
#This Doesn't affect the above descriminator training.     
discriminator.trainable = False  

#This specifies that our Discriminator will take the images generated by our Generator
#and true dataset and set its output to a parameter called valid, which will indicate
#whether the input is real or not.  
valid = discriminator(img)  #Validity check on the generated image


#Here we combined the models and also set our loss function and optimizer. 
#Again, we are only training the generator here. 
#The ultimate goal here is for the Generator to fool the Discriminator.  
# The combined model  (stacked generator and discriminator) takes
# noise as input => generates images => determines validity

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


train(epochs=30000, batch_size=32, save_interval=5000)

#Save model for future use to generate fake images
#Not tested yet... make sure right model is being saved..
#Compare with GAN4

generator.save('generator_model.h5')  #Test the model on GAN4_predict...
#Change epochs back to 30K
                

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

96 [D loss: 0.008437, acc.: 100.00%] [G loss: 4.700516]
97 [D loss: 0.008743, acc.: 100.00%] [G loss: 4.680485]
98 [D loss: 0.014057, acc.: 100.00%] [G loss: 4.774483]
99 [D loss: 0.006464, acc.: 100.00%] [G loss: 4.797379]
100 [D loss: 0.007224, acc.: 100.00%] [G loss: 4.680207]
101 [D loss: 0.014646, acc.: 100.00%] [G loss: 4.745208]
102 [D loss: 0.008167, acc.: 100.00%] [G loss: 4.646924]
103 [D loss: 0.008012, acc.: 100.00%] [G loss: 4.511730]
104 [D loss: 0.024045, acc.: 100.00%] [G loss: 5.050385]
105 [D loss: 0.010294, acc.: 100.00%] [G loss: 4.962698]
106 [D loss: 0.018671, acc.: 100.00%] [G loss: 4.815865]
107 [D loss: 0.017816, acc.: 100.00%] [G loss: 4.811772]
108 [D loss: 0.012773, acc.: 100.00%] [G loss: 4.902825]
109 [D loss: 0.012938, acc.: 100.00%] [G loss: 5.264592]
110 [D loss: 0.012286, acc.: 100.00%] [G loss: 5.236553]
111 [D loss: 0.007451, acc.: 100.00%] [G loss: 5.314220]
112 [D loss: 0.005222, acc.: 100.00%] [G loss: 5.131656]
113 [D loss: 0.004796, acc.: 100.00

244 [D loss: 0.543783, acc.: 75.00%] [G loss: 1.666074]
245 [D loss: 0.694965, acc.: 53.12%] [G loss: 1.015264]
246 [D loss: 0.570357, acc.: 65.62%] [G loss: 1.578037]
247 [D loss: 0.822400, acc.: 43.75%] [G loss: 1.105094]
248 [D loss: 0.622611, acc.: 56.25%] [G loss: 1.457451]
249 [D loss: 0.969546, acc.: 34.38%] [G loss: 0.798251]
250 [D loss: 0.459235, acc.: 71.88%] [G loss: 1.416067]
251 [D loss: 0.873412, acc.: 37.50%] [G loss: 1.103684]
252 [D loss: 0.609774, acc.: 62.50%] [G loss: 1.077602]
253 [D loss: 0.576399, acc.: 68.75%] [G loss: 1.197147]
254 [D loss: 0.804697, acc.: 46.88%] [G loss: 1.057196]
255 [D loss: 0.803893, acc.: 43.75%] [G loss: 0.754151]
256 [D loss: 0.630985, acc.: 56.25%] [G loss: 1.102896]
257 [D loss: 0.716717, acc.: 53.12%] [G loss: 1.049771]
258 [D loss: 0.622084, acc.: 65.62%] [G loss: 1.052108]
259 [D loss: 0.829076, acc.: 34.38%] [G loss: 0.803755]
260 [D loss: 0.543364, acc.: 68.75%] [G loss: 1.314382]
261 [D loss: 0.836201, acc.: 43.75%] [G loss: 0.

393 [D loss: 0.689290, acc.: 40.62%] [G loss: 0.665772]
394 [D loss: 0.665594, acc.: 50.00%] [G loss: 0.655669]
395 [D loss: 0.669869, acc.: 50.00%] [G loss: 0.674545]
396 [D loss: 0.632636, acc.: 56.25%] [G loss: 0.696337]
397 [D loss: 0.652014, acc.: 46.88%] [G loss: 0.682273]
398 [D loss: 0.675596, acc.: 46.88%] [G loss: 0.662492]
399 [D loss: 0.681186, acc.: 50.00%] [G loss: 0.675404]
400 [D loss: 0.656265, acc.: 50.00%] [G loss: 0.654483]
401 [D loss: 0.661505, acc.: 46.88%] [G loss: 0.650516]
402 [D loss: 0.640317, acc.: 56.25%] [G loss: 0.669532]
403 [D loss: 0.672177, acc.: 43.75%] [G loss: 0.672531]
404 [D loss: 0.690825, acc.: 46.88%] [G loss: 0.681718]
405 [D loss: 0.674817, acc.: 53.12%] [G loss: 0.669798]
406 [D loss: 0.665711, acc.: 53.12%] [G loss: 0.663339]
407 [D loss: 0.675755, acc.: 53.12%] [G loss: 0.678877]
408 [D loss: 0.699322, acc.: 50.00%] [G loss: 0.696105]
409 [D loss: 0.667241, acc.: 43.75%] [G loss: 0.698830]
410 [D loss: 0.651988, acc.: 53.12%] [G loss: 0.

540 [D loss: 0.697646, acc.: 56.25%] [G loss: 0.700707]
541 [D loss: 0.693786, acc.: 53.12%] [G loss: 0.736072]
542 [D loss: 0.639024, acc.: 68.75%] [G loss: 0.732015]
543 [D loss: 0.695719, acc.: 56.25%] [G loss: 0.733909]
544 [D loss: 0.681364, acc.: 56.25%] [G loss: 0.708729]
545 [D loss: 0.633486, acc.: 71.88%] [G loss: 0.694180]
546 [D loss: 0.650056, acc.: 62.50%] [G loss: 0.693257]
547 [D loss: 0.646842, acc.: 56.25%] [G loss: 0.693222]
548 [D loss: 0.632313, acc.: 62.50%] [G loss: 0.705319]
549 [D loss: 0.644720, acc.: 62.50%] [G loss: 0.694731]
550 [D loss: 0.626505, acc.: 62.50%] [G loss: 0.704998]
551 [D loss: 0.674977, acc.: 56.25%] [G loss: 0.689329]
552 [D loss: 0.669917, acc.: 59.38%] [G loss: 0.688598]
553 [D loss: 0.678314, acc.: 59.38%] [G loss: 0.702169]
554 [D loss: 0.635838, acc.: 59.38%] [G loss: 0.721573]
555 [D loss: 0.636157, acc.: 75.00%] [G loss: 0.721392]
556 [D loss: 0.670120, acc.: 53.12%] [G loss: 0.723602]
557 [D loss: 0.637862, acc.: 59.38%] [G loss: 0.

...

29328 [D loss: 0.696950, acc.: 56.25%] [G loss: 0.798304]
29329 [D loss: 0.638800, acc.: 56.25%] [G loss: 0.872322]
29330 [D loss: 0.667828, acc.: 68.75%] [G loss: 0.825028]
29331 [D loss: 0.654497, acc.: 62.50%] [G loss: 0.831796]
29332 [D loss: 0.598556, acc.: 68.75%] [G loss: 0.910918]
29333 [D loss: 0.763151, acc.: 53.12%] [G loss: 0.821617]
29334 [D loss: 0.632855, acc.: 65.62%] [G loss: 0.817225]
29335 [D loss: 0.665250, acc.: 62.50%] [G loss: 0.823047]
29336 [D loss: 0.654011, acc.: 59.38%] [G loss: 0.819306]
29337 [D loss: 0.701192, acc.: 43.75%] [G loss: 0.827516]
29338 [D loss: 0.686687, acc.: 56.25%] [G loss: 0.865720]
29339 [D loss: 0.649994, acc.: 68.75%] [G loss: 0.820554]
29340 [D loss: 0.672743, acc.: 59.38%] [G loss: 0.843188]
29341 [D loss: 0.709662, acc.: 53.12%] [G loss: 0.830695]
29342 [D loss: 0.659956, acc.: 59.38%] [G loss: 0.834698]
29343 [D loss: 0.692460, acc.: 50.00%] [G loss: 0.792611]
29344 [D loss: 0.679546, acc.: 53.12%] [G loss: 0.871695]
29345 [D loss:

29471 [D loss: 0.602569, acc.: 68.75%] [G loss: 0.745878]
29472 [D loss: 0.684195, acc.: 43.75%] [G loss: 0.826988]
29473 [D loss: 0.696845, acc.: 46.88%] [G loss: 0.759168]
29474 [D loss: 0.756381, acc.: 56.25%] [G loss: 0.767800]
29475 [D loss: 0.690526, acc.: 50.00%] [G loss: 0.838105]
29476 [D loss: 0.817575, acc.: 43.75%] [G loss: 0.829533]
29477 [D loss: 0.630252, acc.: 68.75%] [G loss: 0.817380]
29478 [D loss: 0.598146, acc.: 68.75%] [G loss: 0.873974]
29479 [D loss: 0.637276, acc.: 59.38%] [G loss: 0.800276]
29480 [D loss: 0.772332, acc.: 37.50%] [G loss: 0.790678]
29481 [D loss: 0.712729, acc.: 40.62%] [G loss: 0.791892]
29482 [D loss: 0.679751, acc.: 56.25%] [G loss: 0.796103]
29483 [D loss: 0.693027, acc.: 53.12%] [G loss: 0.778408]
29484 [D loss: 0.683310, acc.: 59.38%] [G loss: 0.750503]
29485 [D loss: 0.652535, acc.: 65.62%] [G loss: 0.769670]
29486 [D loss: 0.643417, acc.: 62.50%] [G loss: 0.780882]
29487 [D loss: 0.700892, acc.: 53.12%] [G loss: 0.878352]
29488 [D loss:

29613 [D loss: 0.698848, acc.: 53.12%] [G loss: 0.857039]
29614 [D loss: 0.647920, acc.: 59.38%] [G loss: 0.919634]
29615 [D loss: 0.727340, acc.: 46.88%] [G loss: 0.925340]
29616 [D loss: 0.593617, acc.: 75.00%] [G loss: 0.887932]
29617 [D loss: 0.800090, acc.: 34.38%] [G loss: 0.862911]
29618 [D loss: 0.631149, acc.: 62.50%] [G loss: 0.844123]
29619 [D loss: 0.713981, acc.: 59.38%] [G loss: 0.821265]
29620 [D loss: 0.619523, acc.: 68.75%] [G loss: 0.835441]
29621 [D loss: 0.735900, acc.: 46.88%] [G loss: 0.859093]
29622 [D loss: 0.717894, acc.: 53.12%] [G loss: 0.871351]
29623 [D loss: 0.697431, acc.: 56.25%] [G loss: 0.891567]
29624 [D loss: 0.672212, acc.: 59.38%] [G loss: 0.887525]
29625 [D loss: 0.626419, acc.: 62.50%] [G loss: 0.933355]
29626 [D loss: 0.602187, acc.: 68.75%] [G loss: 0.831615]
29627 [D loss: 0.597905, acc.: 68.75%] [G loss: 0.908588]
29628 [D loss: 0.775701, acc.: 53.12%] [G loss: 0.814746]
29629 [D loss: 0.723506, acc.: 40.62%] [G loss: 0.781341]
29630 [D loss:

29755 [D loss: 0.758149, acc.: 50.00%] [G loss: 0.875441]
29756 [D loss: 0.721604, acc.: 50.00%] [G loss: 0.857073]
29757 [D loss: 0.716477, acc.: 53.12%] [G loss: 0.895352]
29758 [D loss: 0.735987, acc.: 43.75%] [G loss: 0.933947]
29759 [D loss: 0.633343, acc.: 68.75%] [G loss: 0.859889]
29760 [D loss: 0.633322, acc.: 65.62%] [G loss: 0.999422]
29761 [D loss: 0.729439, acc.: 50.00%] [G loss: 0.904754]
29762 [D loss: 0.672100, acc.: 68.75%] [G loss: 0.959604]
29763 [D loss: 0.712479, acc.: 50.00%] [G loss: 0.814277]
29764 [D loss: 0.716527, acc.: 56.25%] [G loss: 0.791903]
29765 [D loss: 0.691607, acc.: 53.12%] [G loss: 0.844665]
29766 [D loss: 0.684158, acc.: 53.12%] [G loss: 0.847630]
29767 [D loss: 0.665405, acc.: 62.50%] [G loss: 0.783757]
29768 [D loss: 0.621561, acc.: 65.62%] [G loss: 0.842759]
29769 [D loss: 0.649423, acc.: 53.12%] [G loss: 0.788763]
29770 [D loss: 0.593545, acc.: 71.88%] [G loss: 0.914621]
29771 [D loss: 0.669819, acc.: 59.38%] [G loss: 0.891059]
29772 [D loss:

29897 [D loss: 0.658343, acc.: 71.88%] [G loss: 0.797108]
29898 [D loss: 0.613087, acc.: 68.75%] [G loss: 0.805829]
29899 [D loss: 0.597069, acc.: 65.62%] [G loss: 0.816985]
29900 [D loss: 0.676731, acc.: 56.25%] [G loss: 0.846838]
29901 [D loss: 0.633989, acc.: 65.62%] [G loss: 0.767338]
29902 [D loss: 0.756262, acc.: 50.00%] [G loss: 0.841855]
29903 [D loss: 0.689009, acc.: 62.50%] [G loss: 0.731512]
29904 [D loss: 0.651842, acc.: 68.75%] [G loss: 0.752641]
29905 [D loss: 0.742613, acc.: 50.00%] [G loss: 0.861772]
29906 [D loss: 0.644980, acc.: 65.62%] [G loss: 0.778865]
29907 [D loss: 0.716597, acc.: 50.00%] [G loss: 0.864004]
29908 [D loss: 0.621093, acc.: 62.50%] [G loss: 0.827048]
29909 [D loss: 0.694934, acc.: 62.50%] [G loss: 0.848676]
29910 [D loss: 0.705006, acc.: 50.00%] [G loss: 0.763642]
29911 [D loss: 0.640381, acc.: 68.75%] [G loss: 0.856839]
29912 [D loss: 0.623202, acc.: 65.62%] [G loss: 0.814971]
29913 [D loss: 0.710221, acc.: 50.00%] [G loss: 0.823267]
29914 [D loss: